<a href="https://colab.research.google.com/github/manishsinha44/Machine-Learning/blob/main/AutoEncoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Getting the data Movie Lens ML-100k

In [4]:
!wget "http://files.grouplens.org/datasets/movielens/ml-100k.zip"
!unzip ml-100k.zip
!ls

--2022-01-30 04:51:23--  http://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip.3’

ml-100k.zip.3       100%[===================>]   4.70M  27.3MB/s    in 0.2s    

2022-01-30 04:51:23 (27.3 MB/s) - ‘ml-100k.zip.3’ saved [4924029/4924029]

Archive:  ml-100k.zip
   creating: ml-100k/
  inflating: ml-100k/allbut.pl       
   creating: __MACOSX/
   creating: __MACOSX/ml-100k/
  inflating: __MACOSX/ml-100k/._allbut.pl  
  inflating: ml-100k/mku.sh          
  inflating: __MACOSX/ml-100k/._mku.sh  
  inflating: ml-100k/README          
  inflating: __MACOSX/ml-100k/._README  
  inflating: ml-100k/u.data          
  inflating: __MACOSX/ml-100k/._u.data  
  inflating: ml-100k/u.genre         
  inflating: __MACOSX/ml-100k/.

## Getting ML-1M dataset

In [5]:
!wget "http://files.grouplens.org/datasets/movielens/ml-1m.zip"
!unzip ml-1m.zip
!ls

--2022-01-30 04:51:36--  http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip.1’

ml-1m.zip.1         100%[===================>]   5.64M  31.2MB/s    in 0.2s    

2022-01-30 04:51:37 (31.2 MB/s) - ‘ml-1m.zip.1’ saved [5917549/5917549]

Archive:  ml-1m.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of ml-1m.zip or
        ml-1m.zip.zip, and cannot find ml-1m.zip.ZIP, period.
__MACOSX  ml-100k.zip	 ml-100k.zip.2	ml-1m.zip    sample_data
ml-100k   ml-100k.zip.1  ml-100k

## Importing the Libraries

In [6]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

## Preparing the training set and the test set

In [7]:
training_set = pd.read_csv('ml-100k/u1.base', delimiter = '\t')
training_set = np.array(training_set, dtype = 'int') # Coverting to array to be used with Pytorch 
test_set = pd.read_csv('ml-100k/u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')

## Getting the number of users and movies

In [8]:
# This is needed so that we can create a matrix of users and movies in rows and columns respectively
nb_users = int(max(max(training_set[:, 0], ), max(test_set[:, 0])))
nb_movies = int(max(max(training_set[:, 1], ), max(test_set[:, 1])))

In [9]:
nb_movies

1682

In [10]:
nb_users

943

## Convert the data into a matrix/array with users in rows and movies in columns

In [11]:
def convert(data):
  new_data = []
  for id_users in range(1, nb_users + 1):
    id_movies = data[:, 1] [data[:, 0] == id_users]
    id_ratings = data[:, 2] [data[:, 0] == id_users]
    ratings = np.zeros(nb_movies)
    ratings[id_movies - 1] = id_ratings
    new_data.append(list(ratings))
  return new_data
training_set = convert(training_set)
test_set = convert(test_set)

## Converting data to Torch Sensors  

In [12]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

## Creating the architecture of the Neural Network

In [13]:
class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)

## Training the SAE

In [14]:
nb_epoch = 200
for epoch in range(1, nb_epoch + 1):
  train_loss = 0
  s = 0.
  for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = input.clone()
    if torch.sum(target.data > 0) > 0:
      output = sae(input)
      target.require_grad = False
      output[target == 0] = 0
      loss = criterion(output, target)
      mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
      loss.backward()
      train_loss += np.sqrt(loss.data*mean_corrector)
      s += 1.
      optimizer.step()
  print('epoch: '+str(epoch)+'loss: '+ str(train_loss/s))

epoch: 1loss: tensor(1.7710)
epoch: 2loss: tensor(1.0967)
epoch: 3loss: tensor(1.0533)
epoch: 4loss: tensor(1.0383)
epoch: 5loss: tensor(1.0309)
epoch: 6loss: tensor(1.0264)
epoch: 7loss: tensor(1.0240)
epoch: 8loss: tensor(1.0222)
epoch: 9loss: tensor(1.0208)
epoch: 10loss: tensor(1.0197)
epoch: 11loss: tensor(1.0190)
epoch: 12loss: tensor(1.0185)
epoch: 13loss: tensor(1.0178)
epoch: 14loss: tensor(1.0175)
epoch: 15loss: tensor(1.0175)
epoch: 16loss: tensor(1.0168)
epoch: 17loss: tensor(1.0169)
epoch: 18loss: tensor(1.0163)
epoch: 19loss: tensor(1.0165)
epoch: 20loss: tensor(1.0163)
epoch: 21loss: tensor(1.0161)
epoch: 22loss: tensor(1.0161)
epoch: 23loss: tensor(1.0160)
epoch: 24loss: tensor(1.0157)
epoch: 25loss: tensor(1.0159)
epoch: 26loss: tensor(1.0155)
epoch: 27loss: tensor(1.0155)
epoch: 28loss: tensor(1.0150)
epoch: 29loss: tensor(1.0126)
epoch: 30loss: tensor(1.0122)
epoch: 31loss: tensor(1.0094)
epoch: 32loss: tensor(1.0097)
epoch: 33loss: tensor(1.0055)
epoch: 34loss: tens

## Testing the SAE

In [15]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
  input = Variable(training_set[id_user]).unsqueeze(0)
  target = Variable(test_set[id_user]).unsqueeze(0)
  if torch.sum(target.data > 0) > 0:
    output = sae(input)
    target.require_grad = False
    output[target == 0] = 0
    loss = criterion(output, target)
    mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
    test_loss += np.sqrt(loss.data*mean_corrector)
    s += 1.
print('test loss: '+str(test_loss/s))

test loss: tensor(0.9663)
